In [24]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = '1d17b85b-355b-47cb-b352-5f77df519849'

# configure client
pc = Pinecone(api_key=api_key)

In [25]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

In [26]:
import time

index_name = 'books'

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.05965,
 'namespaces': {'': {'vector_count': 5965}},
 'total_vector_count': 5965}

In [27]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [28]:
import pandas as pd

df = pd.read_csv('preprocessed_dataset.csv')
df.tail()

,information,source_file_name,page_number
5960,اﻻﻋراضاﻟﻣرض \n1 . وﺟود ﻣﺷﺎﻛل ﻓﻲ اﻻﺳﺗﯾﻘﺎظ ﺻﺑﺎﺣﺎ...,ذوي_الاحتياجات_الخاصة[1].pdf,39
5961,اﻻﻋراضاﻟﻣرض \n1 . ﻣن زﯾﺎدة وزﻧﮫ ﺣﺗﻰ إن ﻛﺎن ﻧﺣﯾ...,ذوي_الاحتياجات_الخاصة[1].pdf,40
5962,اﻻﻋراضاﻟﻣرض \n1 . اﻻﻧﻌزال أو اﻟﺧوف أو اﻟﺣزن أو...,ذوي_الاحتياجات_الخاصة[1].pdf,41
5963,اﻻﻋراضاﻟﻣرض \n1 .ﻓﻲ اﻟﻘﻠب أو ﺗﺳﺎرع ﻓﻲ ﺿرﺑﺎت اﻟ...,ذوي_الاحتياجات_الخاصة[1].pdf,42
5964,اﻻﻋراضاﻟﻣرض \n1 . واﻟﺗﻠﻌﺛم واﻟﺗﺄﺗﺄهﺗﻛرار اﻷﺻوا...,ذوي_الاحتياجات_الخاصة[1].pdf,43


In [29]:
df.source_file_name.value_counts()

source_file_name
Diagnostic and statistical manual of mental disorders _ DSM-5 ( PDFDrive.com ).pdf    5838
ذوي_الاحتياجات_الخاصة[1].pdf                                                            61
19-mh-8084-autismspectrumdisorder.pdf                                                   29
يوسف علاء .pdf                                                                          21
اضطرابات نفسيه وعقليه .pdf                                                              16
Name: count, dtype: int64

In [30]:
# Create a copy of the dataframe for efficient iteration
data = df[df.source_file_name.isin(['ذوي_الاحتياجات_الخاصة[1].pdf','يوسف علاء .pdf','اضطرابات نفسيه وعقليه .pdf'])].copy()
data.source_file_name[data.source_file_name == 'ذوي_الاحتياجات_الخاصة[1].pdf'] = 'arabic_file_1'   # Avoid modifying the original dataframe
data.source_file_name[data.source_file_name == 'يوسف علاء .pdf'] = 'arabic_file_2'   # Avoid modifying the original dataframe
data.source_file_name[data.source_file_name == 'اضطرابات نفسيه وعقليه .pdf'] = 'arabic_file_3'   # Avoid modifying the original dataframe
data

C:\Users\LAPTOP WORLD\AppData\Local\Temp\ipykernel_960\1078024635.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.source_file_name[data.source_file_name == 'ذوي_الاحتياجات_الخاصة[1].pdf'] = 'arabic_file_1'   # Avoid modifying the original dataframe
C:\Users\LAPTOP WORLD\AppData\Local\Temp\ipykernel_960\1078024635.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.source_file_name[data.source_file_name == 'يوسف علاء .pdf'] = 'arabic_file_2'   # Avoid modifying the original dataframe
C:\Users\LAPTOP WORLD\AppData\Local\Temp\ipykernel_960\1078024635.py:5: SettingWithCopyWarning: 
A value is trying to be set

,information,source_file_name,page_number
5867,اﻟﺤﺮﻛﻪ ﻓﺮط اﺿﻄﺮاب\n:-اﻟﺤﺮﻛﺔ ﻓﺮط أﻋﺮاض\n ا...,arabic_file_3,1
5868,.ﺳﻨﺔ ٧١ ﻟـ ﺳﻨﻮات 3 ﻣﻦ ﻛﻮﻧﺮز اﺧﺘﺒﺎر ﺗﻄﺒﻴﻖ ...,arabic_file_3,2
5869,.أﺻﻐﺮ ﻟﻤﻬﺎم اﻟﻤﻬﺎم ﺗﻘﺴﻴﻢ ﻣﺮاﻋﺎه ﻣﻦ وﻻﺑﺪ ...,arabic_file_3,3
5870,ﻣﻌﻴﻨﻪ ﺻﺤﯿﺔ وﺣﺎﻟﺔ ﻃﺒﯿﺔ ﺑﻤﺸﻜﻠﺔ\n اﻟﻘﯿﺎم ﻣﻦ ...,arabic_file_3,4
5871,:وﻣﻨھﺎ اﻟﻘﻠﻖ، اﺿﻄﺮاﺑﺎت ﻣﻦ ﻣﺨﺘﻠﻔﺔ أﻧﻮاع ﻋ...,arabic_file_3,5
...,...,...,...
5960,اﻻﻋراضاﻟﻣرض \n1 . وﺟود ﻣﺷﺎﻛل ﻓﻲ اﻻﺳﺗﯾﻘﺎظ ﺻﺑﺎﺣﺎ...,arabic_file_1,39
5961,اﻻﻋراضاﻟﻣرض \n1 . ﻣن زﯾﺎدة وزﻧﮫ ﺣﺗﻰ إن ﻛﺎن ﻧﺣﯾ...,arabic_file_1,40
5962,اﻻﻋراضاﻟﻣرض \n1 . اﻻﻧﻌزال أو اﻟﺧوف أو اﻟﺣزن أو...,arabic_file_1,41
5963,اﻻﻋراضاﻟﻣرض \n1 .ﻓﻲ اﻟﻘﻠب أو ﺗﺳﺎرع ﻓﻲ ﺿرﺑﺎت اﻟ...,arabic_file_1,42


In [31]:
# # Import the tqdm library for progress bars
# from tqdm.auto import tqdm

# # Set the batch size for processing data in chunks
# batch_size = 100

# # Iterate through the data in batches
# for i in tqdm(range(0, len(data), batch_size)):  # Display a progress bar

#     # Calculate the end index for the current batch
#     i_end = min(len(data), i + batch_size)

#     # Get the current batch of data from the dataframe
#     batch = data.iloc[i:i_end]

#     # Generate unique IDs using file name, page number, and index within the batch
#     ids = [f"{x['source_file_name']}-{x['page_number']}-{i}" for i, x in batch.iterrows()]

#     # Extract the text to be embedded from the batch
#     texts = [x['information'] for _, x in batch.iterrows()]

#     # Generate embeddings for the text
#     embeds = model.encode(texts).tolist()

#     # Prepare metadata to be stored in Pinecone alongside the embeddings
#     metadata = [
#         {'information': x['information'], 'page_number': x['page_number'], 'source_file_name': x['source_file_name']}
#         for i, x in batch.iterrows()
#     ]

#     # Add the embeddings and metadata to Pinecone in a batch
#     index.upsert(vectors=zip(ids, embeds, metadata))

In [32]:
embeds

[[-0.04951218143105507,
  0.020069094374775887,
  -0.018688127398490906,
  -0.056418608874082565,
  -0.07209836691617966,
  -0.022889066487550735,
  0.024794356897473335,
  -0.006566173862665892,
  0.01155474130064249,
  -0.028569411486387253,
  0.10880228132009506,
  -0.023810941725969315,
  0.023925354704260826,
  -0.0023599599953740835,
  -0.046329937875270844,
  -0.018376562744379044,
  -0.11758940666913986,
  0.022682074457406998,
  -0.08868546783924103,
  0.04036165773868561,
  -0.0014725488144904375,
  0.03879770264029503,
  0.05602141469717026,
  -0.01764054037630558,
  0.031875286251306534,
  -0.012951957993209362,
  -0.00015105563215911388,
  0.11501328647136688,
  0.009699477814137936,
  -0.028460277244448662,
  -0.0011107188183814287,
  0.06874130666255951,
  0.020490143448114395,
  0.0066309645771980286,
  0.028653867542743683,
  0.02648254856467247,
  0.013703528791666031,
  -0.014928523451089859,
  0.04785926640033722,
  0.11223985999822617,
  -0.03481970354914665,
  -0.

In [33]:
def cut_context(context):
    sum_tokens = 0
    context.reverse()
    for i, qna in enumerate(context):
        sum_tokens += len(str(qna).split())
        if sum_tokens > 2048:
            context.reverse()
            return context[i:]
    context.reverse()
    return context

def augment_prompt(query: str, history:list):
    # get top 3 results from knowledge base
    results = cut_context([match['metadata'] for match in index.query(4,model.encode(str(history) + '\n' + query).tolist(),include_metadata=True)['matches']])
    # get the text from the results
    source_knowledge = "\n".join([str(x) for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""You are a Mental Health Professional who helps people with mental disorders like autism and ADHD.\nUsing the contexts and chat history below, answer the query.

    Contexts:
    {source_knowledge}

    Chat history:
    {str(history)}

    Query: {query}"""
    return augmented_prompt

ap = augment_prompt('هل اعاني من التوحد?',[])

print(ap, len(ap.split()))

You are a Mental Health Professional who helps people with mental disorders like autism and ADHD.
Using the contexts and chat history below, answer the query.

    Contexts:
    {'information': 'ﺑﺴﺒﺐ  أو  أﺧﺮى،  أدوﻳﺔ  أو  دواء  ﺗﻨﺎول  ﻋﻦ  ﺗﻨﺘﺞ  ﻻ  أﻋﺮاض  ﻟﺪﻳﻚ \nﻋﻠﻰ ) اﻟﻼإرادﻳﺔ  اﻟﺘﺸﻨﺠﺎت  ﺗﺴﺒﺐ  أن  ﻳﻤﻜﻦ  ﻃﺒﻴﺔ  ﺣﺎﻟﺔ  وﺟﻮد \nاﻟﺘﺎﻟﻲ  اﻟﺪﻣﺎغ  اﻟﺘﻬﺎب  أو  ﻫﻨﺘﻨﻐﺘﻮن  ﻣﺮض  اﻟﻤﺜﺎل،  ﺳﺒﻴﻞ \n(ﻟﻠﻔﻴﺮوﺳﺎت', 'page_number': 4.0, 'source_file_name': 'arabic_file_2'}
{'information': 'اﻟﺘﻮاﻟﻲ  ﻋﻠﻰ  ﺷﻬﺮًا  12 ﻋﻦ ﺗﺰﻳﺪ  ﻻ  ﻟﻤﺪة  ﺣﺎﺿﺮﻳﻦ  ﻳﻜﻮﻧﻮا  أن', 'page_number': 4.0, 'source_file_name': 'arabic_file_2'}
{'information': 'ﺗﺤﺖ  أو  ﻣﺘﻌﺒًﺎ  أو  ﻣﺘﺤﻤﺴًﺎ  اﻟﺸﺨﺺ  ﻳﻜﻮن  ﻋﻨﺪﻣﺎ  أﺳﻮأ  ﺗﻜﻮن  ﻗﺪ \nاﻟﻬﺎﺗﻒ  ﻋﺒﺮ  اﻟﺘﺤﺪث  أو  ﻣﺠﻤﻮﻋﺔ،  أﻣﺎم  اﻟﺘﺤﺪث:  ﻣﺜﻞ  ﺿﻐﻂ ', 'page_number': 1.0, 'source_file_name': 'arabic_file_2'}
{'information': 'ﻳﻤﻜﻦ.  اﻷﻗﻞ  ﻋﻠﻰ  ﻋﺎم  ﻟﻤﺪة  اﻟﻼإرادﻳﺔ  اﻟﺘﺸﻨﺠﺎت  ﻟﺪﻳﻚ  ﻛﺎن \nﻋﺎدة ) اﻟﻴﻮم  ﻓﻲ  ﻣﺮات  ﻋﺪة  اﻟﻼإرادﻳﺔ  اﻟﺘﺸﻨﺠﺎت  ﺗﺤﺪث  أن \nوﻣﺘﻜﺮرة  ﻣﺘﻘﻄﻌﺔ  أو  ﺗﻘﺮﻳﺒًﺎ،  ﻳﻮم  ﻛﻞ(  ﻧﻮﺑﺎت  ﺷﻜﻞ  ﻋﻠﻰ', 'page_nu

In [35]:
import os
os.environ["OPENAI_API_KEY"] = '' # ال OpenAI API key اللي هندفع بيه لOpenAI

from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct") # اسم الموديل اللي هنسخدمه (كل موديل ليه سعره ودرجة ذكاءه)

# from langchain import PromptTemplate

# template = """
# You are a Mental Health Professional who helps people with mental disorders like autism and ADHD. 
# Answer this question: "{ques1}" with a conversational answer
# """ # دي concatenated message 

# prompt = PromptTemplate(
#     input_variables=["ques1"],
#     template=template,
# )


# from langchain.chains import LLMChain
# chain = LLMChain(llm=llm, prompt=prompt) # الchain هتسهل الكود اكتر


chat_history_ = []

def cut_history(history):
    sum_tokens = 0
    history.reverse()
    for i, qna in enumerate(history):
        sum_tokens += len(str(qna).split())
        if sum_tokens > 2048:
            history.reverse()
            return history[i:]
    history.reverse()
    return history

def cut_context(history):
    sum_tokens = 0
    history.reverse()
    for i, qna in enumerate(history):
        sum_tokens += len(str(qna).split())
        if sum_tokens > 2048:
            history.reverse()
            return history[i:]
    history.reverse()
    return history


def chat_(chat_history, question):
  global chat_history_
      
  prompt = augment_prompt(question, str(cut_history(chat_history_[-9:])).replace('{','<').replace('}','>')) # دي concatenated message 
    # prompt = PromptTemplate(
    #     input_variables=["ques1"],
    #     template=template,
    # )
    # chain = LLMChain(llm=llm, prompt=prompt)
        

#   else:
#         template = 'You are a Mental Health Professional who helps people with mental disorders like autism and ADHD.' +'\n Answer this question: "{ques1}" with a conversational answer.' # دي concatenated message 
#         prompt = PromptTemplate(
#             input_variables=["ques1"],
#             template=template,
#         )
#         chain = LLMChain(llm=llm, prompt=prompt)
  print(prompt)
  bot_response = llm(prompt)
      


  response = ""
  chat_history_.append({"Question": question, "Answer": bot_response})

  for letter in ''.join(bot_response):
      response += letter + ""
      yield chat_history + [(question, response)]

chat_(chat_history_,'Have I ADHD?')

<generator object chat_ at 0x000001F1851C4510>

In [ ]:
question = 'Have I ADHD?'
prompt = augment_prompt(question, str(cut_history(chat_history_[-9:])).replace('{','<').replace('}','>')) # دي concatenated message 
    # prompt = PromptTemplate(
    #     input_variables=["ques1"],
    #     template=template,
    # )
    # chain = LLMChain(llm=llm, prompt=prompt)
        

#   else:
#         template = 'You are a Mental Health Professional who helps people with mental disorders like autism and ADHD.' +'\n Answer this question: "{ques1}" with a conversational answer.' # دي concatenated message 
#         prompt = PromptTemplate(
#             input_variables=["ques1"],
#             template=template,
#         )
#         chain = LLMChain(llm=llm, prompt=prompt)
print(prompt)
bot_response = llm(prompt)
    


response = ""
chat_history_.append({"Question": question, "Answer": bot_response})

You are a Mental Health Professional who helps people with mental disorders like autism and ADHD.
Using the contexts and chat history below, answer the query.

    Contexts:
    {'information': 'There may be an elevated likelihood of  obesity among individuals with ADHD', 'page_number': 108.0, 'source_file_name': 'Diagnostic and statistical manual of mental disorders _ DSM-5 ( PDFDrive.com ).pdf'}
{'information': 'ADHD is a neurodevelopmental disorder defined by impairing levels of inattention, dis-\norganization, and/or hyperactivity-impulsivity. Inattention and disorganization entail inabil-\nity to stay on task, seeming not to listen, and lo sing materials, at leve ls that are inconsistent\nwith age or developmental level.  Hyperactivity-impulsivity entails  overactivity, fidgeting, in-\nability to stay seated, intruding into other people’s activities , and inability to wait—symptoms\nthat are excessive for age or developmental level. In childhood, ADHD frequently overlaps\nwith dis

c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
bot_response

'\n\nBased on the information provided, I cannot determine if you have ADHD. ADHD is a neurodevelopmental disorder characterized by symptoms such as inattention, hyperactivity, and impulsivity. It is important to consult with a mental health professional for a proper evaluation and diagnosis. Other factors, such as visual and hearing impairments, metabolic abnormalities, sleep disorders, and nutritional deficiencies, should also be considered as possible influences on ADHD symptoms.'